In [1]:
import pandas as pd
import numpy as np
from pandas_datareader import data, wb
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
#cor plot index use is inconsistant with some other function maybe HRP?

In [1]:
#%run /Users/davidfitzpatrick/Desktop/Codes/Python_codes/Fun_codes



In [ ]:
#%run /Users/davidfitzpatrick/Desktop/Codes/Python_codes/Finance_codes/tools/helpers.ipynb

#read in stock data
#tickers = ['SPY', 'SPYC']
#start = datetime(2020,1,1)
#end = datetime(2021,7,20)
#returns = import_data(tickers,start,end)


#TIP: use join when have dataframes of different lengths and just want to keep the intersection of rows


# plot dividends
    # pd.DataFrame(port["IVV_Div"]/port["IVV"]).resample('AS').sum().plot()

# annualize returns
    # returns.resample("A").apply(lambda x: ((x + 1).cumprod()-1).last("D"))

#for company info
    #si.get_quote_table("SPY", dict_result = False)

# get weekday
    # week['weekday'] = week['Date'].apply(lambda x: x.weekday())
    
#bid ask info 
    #si.get_quote_table("SPY", dict_result = False)

#no_dupes = [x for n, x in enumerate(a) if x not in a[:n]] #print no_dupes

In [ ]:
#cleaned up, all functions are intended to run on dataframes of returns

In [1]:
#EX

#This model assumes the risk free rate is constant over the period

#t = 1 #number to makes std and means equivelent to yearly returns so 12 for monthly, 52 for weekly and 252 for daily
#posLB = 0   #lower bound on position exposure
#posUB = 1  #upper bound on position exposure
#tickers = col
#increments_for_graph = .001

#make sure returns are decimal
#Cov = np.array((returns[tickers].loc[start_year:end_year]/100).cov())
#muf = returns['Cash'].loc[start_year:end_year].mean() /100
#mean_vect = returns[col].loc[start_year:end_year].mean() /100 #this data gives percents as whole numbers


#depends on scipy.optimize.linprog and qpsolvers.solve_qp
def mean_var_opt(posLB, posUB, t, tickers, increments_for_graph, Cov, muf, mean_vect):
    """
    posLB,posUB: upper and lower bounds for individual positions
    t: number to multiply expected returns by to make them yearly
    tickers: list of column names
    increments_for_graph: decreasing this number increases the number of portfolios calculated
    Cov: covariance of returns
    muf: should be a single value
    mean_vect: should be mean of dataframe

    """
    Cov = 2 * np.array(Cov)
    mean_vect = mean_vect * t
    muf  = muf * t
    m = len(tickers)

    from scipy.optimize import linprog
    #set equality constraints. I want my positions to sum to 1
    A_eq = np.array((1,)*m).reshape(1,-1)
    b_eq = [1]

    # set bounds on leverage. We can short but we can not go long
    bounds = np.vstack([(posLB,)*m, (posUB,)*m]).T

    #find min and max return
    min_rtn = mean_vect @ linprog(c=mean_vect, A_eq = A_eq, b_eq = b_eq, bounds=bounds).x
    max_rtn = mean_vect @ linprog(c=-mean_vect, A_eq = A_eq, b_eq = b_eq, bounds=bounds).x

    from qpsolvers import solve_qp
    muP = np.arange(min_rtn * .995,max_rtn * .995, increments_for_graph)
    sdP = np.zeros(len(muP))
    weights = np.zeros([len(muP),m])

    q_vec = np.zeros(m).reshape(-1,)
    G = np.zeros([m,m])
    h = np.zeros(m)
    A = np.vstack([np.array((1,)*m).reshape(1,-1),mean_vect])

    #calculate each optimized portfolio for each mean
    for i in range(len(muP)):
        b = np.array([1,muP[i]])
        weights[i] = solve_qp(P=Cov, q = q_vec, G=G, h=h, A=A, b=b, lb = bounds[:,0], ub = bounds[:,1])
        sdP[i] = np.sqrt(weights[i] @ (Cov/2) @ weights[i]) * np.sqrt(t)
    
    portfolios = pd.DataFrame({'Returns': muP, 'Volatility': sdP,})
    for counter, symbol in enumerate(tickers):
        portfolios[symbol+' weight'] = [w[counter] for w in weights]
    
    portfolios['Sharpe'] = (portfolios['Returns']-muf) / portfolios['Volatility'] #note using monthly returns and vol

    tangent = portfolios.iloc[[portfolios.Sharpe.argmax()]]
    min_var = portfolios.iloc[[portfolios.Volatility.argmin()]]
    efficient_portfolios = portfolios[portfolios['Returns'] >= min_var.Returns.values[0]-.02] #the .02 is arbitrary to see a bit below the min var port too

    return tangent, min_var, efficient_portfolios

In [3]:
#hyperparameter tuning give 1d numpy arrays ex.
#x1 = tested_params_rand[:,0]
#x2 = tested_params_rand[:,1]
#y = np.ravel(sharpe_rand)
def contourplot(x1,x2,y,lvlnum):
    plt.figure(figsize=(18,6))
    plt.tricontourf(x1, x2, y, levels=lvlnum, cmap="seismic")
    plt.scatter(x1, x2, alpha = .9, c = y, cmap = 'seismic')
    cbar = plt.colorbar()
    plt.show()

In [4]:
#note: bayes opt does better if function being evaluated is in the range of 1-1000
#note: if mean error is high increase initial sample size

#bayes opt
#from numpy.random import normal, uniform, random
from sklearn.gaussian_process import GaussianProcessRegressor
from numpy.random import uniform
from scipy.stats import norm
import time


#from sklearn.svm import SVC
#def func_approx(x):
#    g = x[0]
#    w = x[1]
#    real_model = SVC(gamma = g/10, class_weight={0: w, 1: 1}) #'balanced'
#    real_model.fit(xTr, np.ravel(yTr))
#    predictions = real_model.predict(xVal)
#    risk_layer_performance = np.multiply(y[train_val_threshold:val_test_threshold],predictions)
#    series = pd.Series(risk_layer_performance)
#    geo_rtn = geometric_return(series).values[0]
#    std = series.std()    
#    return geo_rtn/(std+.000001)

#sample of features(N) which sum to 1. Sample M times
#def constrained_samp(M):
#    U=np.zeros([M,2])
#    U[:,0] = uniform(1,1000,M)
#    U[:,1] = uniform(1,50,M)
#    return(U.astype(int))



#sample of features(N) which sum to 1. Sample M times
from numpy.random import uniform
#def constrained_samp(M,N):
#    H=np.zeros([M,N+1])
#    U=np.zeros([M,N])
#    for j in range (0,M):
#        for i in range(1,N): 
#            H[j,i] = round(uniform(0,1),2)
#        H[j,N] = 1
#        H.sort()
#        for i in range(1,N+1):
#            U[j,i-1] = H[j,i] - H[j,i-1]
#    return(U)

# probability of improvement acquisition function
def acquisition(X, Xsamples, model):
    # calculate the best surrogate score found so far
    yhat = model.predict(X)
    best = max(yhat)
    # calculate mean and stdev via surrogate function
    mu, std = model.predict(Xsamples, return_std=True)
    mu = mu.reshape(-1,)
    std = std.reshape(-1,)
    # calculate the probability of improvement
    probs = norm.cdf((mu - best) / (std+1E-9))
    return probs

# optimize the acquisition function
def opt_acquisition(X, y, model,search_samp):
    # random search, generate random samples
    Xsamples = constrained_samp(search_samp)
    # calculate the acquisition function for each sample
    scores = acquisition(X, Xsamples, model)
    # locate the index of the largest scores
    ix = np.argmax(scores)
    return Xsamples[ix]

def bayes_opt(starting_data = 50, search_samp = 100, optimization_steps = 50):
    """
    startig data: number of actual points to start with
    search samp: number of points to sample at each optimization step
    optimization_steps: number of updates
    """
    
    t0 = time.time()

    #samples
    X = constrained_samp(starting_data)
    ys = np.asarray([func_approx(x) for x in X]).reshape(-1,1) ####
    # define the model
    model = GaussianProcessRegressor()
    model.fit(X, ys)

    # perform the optimization process
    error=[]
    for i in range(optimization_steps):
        # select the next point to sample
        point = opt_acquisition(X, ys, model, search_samp)  #####
        # sample the point
        actual = func_approx(point) ####
        # summarize the finding
        est = model.predict(point.reshape(1,-1))
        wrongness=abs(actual-est)/(est+.0001)
        error.append(wrongness.item())
        # add the data to the dataset
        X = np.vstack((X, point))
        ys = np.vstack((ys, [[actual]]))
        # update the model
        model.fit(X, ys)
    
    # best result
    X = np.round(X,2)
    ix = np.argmax(ys)
    print('Best Result: \nWeights=%s, \n function max=%.6f' % (tuple(X[ix]), ys[ix]))
    print('\nMean Error=%.3f' % (np.mean(error)))
    t1 = time.time()
    print("time",t1-t0)
    return(X,ys,ix,error)

In [ ]:
#note intervals might behave oddly if using an extremely volitile asset
#returns dataframe, array and plot

def conditional_performance(df,independant_var, intervals=[.02,.01,.005,0,-.005,-.01,-.02], relative_performance=False, plot = True):
    select_intervals = intervals.copy()
    select_intervals.insert(0,1)
    select_intervals.append(-1)
    
    ave_rtn_in_interval = []
    
    outperformance, probabilities = np.zeros([len(select_intervals)-1,len(df.columns)-1]), np.zeros([len(select_intervals)-1])
    
    for i in range(1,len(select_intervals)):
        df_conditioned = df[(df[independant_var]<select_intervals[i-1]) & (df[independant_var]>select_intervals[i])]
        ave_rtn_in_interval.append(df_conditioned[independant_var].mean())
        probabilities[i-1] = len(df[(df[independant_var]<select_intervals[i-1]) & (df[independant_var]>select_intervals[i])])/len(df)
        if relative_performance == True:
            outperformance[i-1] = np.array( df_conditioned.sub(df_conditioned[independant_var],axis=0).drop([independant_var],axis=1).mean() )
        else:
            outperformance[i-1] = np.array( df_conditioned.drop([independant_var],axis=1).mean() )
            
    
    cond_rtns = pd.DataFrame(outperformance, index = ave_rtn_in_interval, columns = returns.columns.drop(independant_var))
    
    if plot == True:
        for i in range(len(cond_rtns.columns)):
            plt.scatter(cond_rtns.index,cond_rtns[cond_rtns.columns[i]], label="{} data".format(cond_rtns.columns[i]))
        for i in intervals:
            plt.axvline(x=i)
        plt.legend(bbox_to_anchor=(1, 1))
        plt.xlabel(independant_var+' daily return')
        plt.ylabel('daily return')
        plt.title("asset returns conditional on "+independant_var+" return")

    return cond_rtns, probabilities

In [ ]:
def crosscorr(leader, follower, horiz_graph_bounds, plot=False, title="Title"): 

    x = np.arange(len(leader))
    y1 = follower
    y2 = leader
    npts = len(y1)

    lags = np.arange(-npts + 1, npts)
    ccov = np.correlate(y1 - y1.mean(), y2 - y2.mean(), mode='full')
    ccor = ccov / (npts * y1.std() * y2.std())

    #fig, axs = plt.subplots(nrows=2)    
    #fig.subplots_adjust(hspace=0.4)   
    #ax = axs[0]   
    #ax.plot(x, y1, 'b', label='Follower')
    #ax.plot(x, y2, 'r', label='Leader')    
    #ax.set_ylim(-.1, 1)    
    #ax.legend(loc='upper right', fontsize='small', ncol=2)

    if(plot==True):
        fig, axs = plt.subplots(nrows=1)
        ax = axs#[1]
        ax.plot(lags[horiz_graph_bounds:-horiz_graph_bounds], ccor[horiz_graph_bounds:-horiz_graph_bounds]) #npts-1, npts-1
        ax.set_ylim(-.7, .9)
        ax.set_ylabel('Correlation')
        ax.set_xlabel('Days between Leader and Follower')
        plt.title(title)
        plt.show()

    maxlag = np.argmax(ccor[:])#npts-1
    maxcorr = np.max(ccor[:])#npts-1
    print(title,": Max correlation is at lag", maxlag-npts+1,"\nCorrelation is", round(maxcorr,2)) 
    # from following https://currents.soest.hawaii.edu/ocn_data_analysis/_static/SEM_EDOF.html

In [ ]:
#growth of 1 dollar invested
def dollar_growth(returns):
    return ((returns+1).cumprod())

In [ ]:
#Calculate geometric return from imported return data
def geometric_return(data):
    return ((data+1).cumprod()[-1:].abs()**(1/(len(data)))-1)

In [ ]:
def import_data(tickers, start, end, give = "R", adj_close = True, dividend = True):#read in stock data  
    if give=='D':
        adj_close=False
        dividend = True
    
    stocks = pd.DataFrame()
    if (adj_close == True):
        dividend = False
        stocks[tickers] = data.DataReader(tickers, 'yahoo', start, end)['Adj Close']
    else:
        stocks[tickers] = data.DataReader(tickers, 'yahoo', start, end)['Close']
            
        if (dividend == True): 
            stock_div = pd.DataFrame(index=stocks.index)
            for tick in tickers:
                stock_div[tick+' div'] = data.DataReader(tick, 'yahoo-dividends', start, end)["value"]
      
    just_stocks=stocks
    if (dividend == True):
        stocks = pd.concat([stocks,stock_div],axis=1)
        stocks = stocks.fillna(0)
        
    if(give == "R"):
        #Adding dividends to price appreciation on a daily percent return basis -useful for all analysis
        raw_returns=pd.DataFrame()
        for tick in tickers:
            if (dividend == True):
                raw_returns[tick] = stocks[tick].pct_change() + stocks[tick+' div']/stocks[tick] 
                #dividends are actually added on the day before they should be added but this error is neglegible
            else:
                raw_returns[tick] = stocks[tick].pct_change()
        returns=raw_returns[1:]
        return returns[np.isfinite(returns).all(1)]
        
    if(give == "P"):
        return just_stocks
    
    if(give=="D"):
        dividend = pd.DataFrame(np.array(stocks.iloc[:,len(just_stocks.columns):])/np.array(stocks.iloc[:,:len(just_stocks.columns)])).set_index(just_stocks.index)
        dividend.columns=just_stocks.columns
        return dividend
    
    #importing financial data from yahoo finance
#commodity_futures = ['GC=F', 'SI=F', 'CL=F']
#cryptocurrencies = ['BTC-USD', 'ETH-USD', 'XRP-USD']
#currencies = ['EURUSD=X', 'JPY=X', 'GBPUSD=X']
#mutual_funds = ['PRLAX', 'QASGX', 'HISFX']
#us_treasuries = ['^TNX', '^IRX', '^TYX']

In [ ]:
def yield_curve(start,end,plot=True):
    
    if (plot==True):
        Interest_rates = ['^IRX','^TNX'] # '^FVX', '^TYX'
        IR = import_data(Interest_rates,start,end, give ="P", dividend=True)
        IR["yd_cve"]=IR["^TNX"]-IR["^IRX"]
        IR["yd_cve"].plot()
        plt.title("10 year - 3 month rate")
        plt.show(block=True)
    if (plot==False):
        Interest_rates = ['^IRX','^FVX','^TNX','^TYX']
        IR = import_data(Interest_rates,start,end, give ="P", dividend=True)
        return IR

In [2]:
from pandas.core.common import flatten
#from diversified ports
#backtest shell function
def backtest(weights, col, dataf, days_between_rebalance, rebalance_func, show_weights=False, wealth = 1, RebalanceOffset=0, give='R',formation = 0): #cols allows users to backtest subsets of dataframe
    """
    weights: tells the function the number of assets. sets the starting weights and is the weights for constant rebalancing
    wealth: sets starting value
    RebalanceOffset: to offset day of rebalances by these values to avoid rebalance timing luck
    give: return returns or prices
    formation: number of days covariance matrix is formed over
    
    """
    colin=[0]*len(weights)
    port=[0]*len(weights)
    worth=[wealth]

    for n in range(len(weights)):
        colin[n] = dataf.columns.get_loc(col[n])
        port[n]=wealth*weights[n]

    for n in range (formation,len(dataf)):
        port = np.multiply(port, np.array((1+dataf.iloc[n,colin])))

        end_of_day = sum(port)
        worth.append(end_of_day)
        
        if ((n + RebalanceOffset)% days_between_rebalance==0):        #rebalance
            port=rebalance_func(dataf, n, formation, port, weights, sum(port))
            if show_weights == True:
                print(np.round(np.array(port/sum(port)),2))
            
                
    if (give=='R'):            
        p_rtn = pd.DataFrame(worth).pct_change()[1:].set_index(dataf.index[formation:])
        return(p_rtn)
    if (give=='P'):
        return(pd.DataFrame(worth[1:]).set_index(dataf.index[formation:]))
    
def constant_weight_rebalance(stocks, n, formation_period, port, weights, total):  
   #intended for monthly rebalancing
    update = np.subtract(np.array(weights), np.array(port/total))/4
    new_port = np.add(np.array(port/total),update)    #update portfolio weights
    #print(np.round(new_port,2),n)                       #print weights
    return total * np.array(weights)

def ivp_rebalance(stocks, n, formation_period, port, weights, total):
    cov = stocks[n-formation_period : n - 1].cov() 
    ivp = list(flatten(getIVP(cov).reshape(-1,1)))
    return total * np.array(ivp)

def hrp_rebalance(stocks, n, formation_period, port, weights, total):
    formation = stocks[n - formation_period : n - 1]
    cov,corr = formation.cov(),formation.corr()     #2) compute correl matrix           
    
    dist=correlDist(corr)                #3) cluster
    link=sch.linkage(dist,'single')
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist()   # recover labels
    hrp=getRecBipart(cov,sortIx)         #allocation  
    hrp = np.array(hrp[stocks.columns])  #sort
    return total * hrp


from scipy.spatial.distance import squareform
def hrp_rebalance(stocks, n, formation_period, port, weights, total):
    #formation period is 252 * 2
    #hrp recursively bisect and equalize risk budgets at every level of heirarchy
    formation = stocks[n - formation_period : n - 1]
    cov,corr = formation.cov(),formation.corr()     #2) compute correl matrix       
    dist=correlDist(corr)                #3) cluster
    link=sch.linkage(squareform(dist),'single')   
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist()   # recover labels
    hrp=getRecBipart(cov,sortIx,formation.mean()) # allocation 

    #for rebalancing every month (21 business days), uses sorted HRP
    update = np.subtract(np.array(hrp[stocks.columns]), np.array(port/total))/4   #reduce turnover and rebalance timing luck by only moving towards optimal value
    
    #print(abs(update).sum()/2)                       #print turnover
    new_port = np.add(np.array(port/total),update)    #update portfolio weights
    #print(np.round(new_port,2),n)                       #print weights
    return total * new_port

In [ ]:
#to clean

In [3]:
from numpy.lib.stride_tricks import as_strided

def windowed_view(x, window_size):
    """Creat a 2d windowed view of a 1d array.

    `x` must be a 1d numpy array.

    `numpy.lib.stride_tricks.as_strided` is used to create the view.
    The data is not copied.

    Example:

    >>> x = np.array([1, 2, 3, 4, 5, 6])
    >>> windowed_view(x, 3)
    array([[1, 2, 3],
           [2, 3, 4],
           [3, 4, 5],
           [4, 5, 6]])
    """
    y = as_strided(x, shape=(x.size - window_size + 1, window_size),
                   strides=(x.strides[0], x.strides[0]))
    return y

def max_dd(prices):
    cum_returns = (1 + prices).cumprod()
    drawdown =  1 - prices.div(prices.cummax())
    return drawdown.expanding().max().iloc[-1,:]

In [ ]:
def Turbulence_indicator(returns): #reduces dataset by 15 days because covariance matrix is unstable before that
    turbulence=[]
    for i in range(15,len(returns)):
        deviation = returns.iloc[i]-returns[:i].mean() #[:i] prevents information bleed
        inverse = np.linalg.inv(returns[:i].cov())     #[:i] prevents information bleed
        turb = np.sqrt(np.matmul(np.matmul(deviation.T,inverse),deviation))
        turbulence.append(turb)
    pd.options.mode.chained_assignment = None  # default='warn'
    AllData = returns[15:]
    AllData['Turbulence']=turbulence
    pd.options.mode.chained_assignment = 'warn'  # default='warn'
    return AllData

#Turbulence indicator- I use the square root because the magnitude of the indicator is higher for daily data
#starts 15 days in for stability reasons. inverse covariance matrix is large and sometimes negative for small sample sizes

In [ ]:
def beta(regressor, data, start, end): #Monthly beta
    from sklearn.linear_model import LinearRegression
    #from sklearn.base import clone
    #import sklearn
    lm = LinearRegression()
    riskfree = 'VFISX' #VFISX is a poor risk_free rate maturity currently being 3 years, but it's convenient
    #returns = import_data([regressor,riskfree]+tickers,start,end, give ="R",dividend=True) alt method
    returns_internal = import_data([regressor,riskfree],start,end, give ="R",dividend=True)
    all_data = returns_internal.join(data,how='inner')
    for col in (all_data.columns):
        all_data[col] = all_data[col]-all_data[riskfree]
    adjusted = all_data.drop(riskfree,axis=1).resample("M").apply(lambda x: ((x + 1).cumprod()-1).last("D"))
    
    coef=[]
    for col in (adjusted.columns[1:]):
        X = np.array(adjusted[regressor]).reshape(-1,1)
        y = np.array(adjusted[col]).reshape(-1,1)
        lm.fit(X,y) #use all data as training data because we arn't predicting anything
        coef.append(lm.coef_[0])
        
    coeff_df = pd.DataFrame(coef,columns=["Beta"]).set_index(adjusted.columns[1:]) 
    print(coeff_df)

In [2]:
#plotting the monthly time series of correlation

def cor_plot(start_year, end_year, dataset, SandP,lines_per_plot,num_plots,lower_y):
    cor = pd.DataFrame()
    month = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec"]
    i=0
    j=0
    for j in np.arange(start_year,end_year+1):
        for i in np.arange(1,13):
            cor[month[i-1]+str(j)] = correlation(dataset,SandP,datetime(j,i,1), datetime(j,i,28))       
    for i in np.arange(0,num_plots,1):
        print(cor[i:i+lines_per_plot].T.rolling(4).mean().plot().set_ylim(lower_y, 1))
        
def correlation(dataset,SandP,start,end):
    return dataset[start:end].corr()[SandP]

In [ ]:
def SMA_Backtest(SMA_len, return_len, data, initial):
    timing = initial
    hold = 'cash'
    acct_value = []
    trades=[]
    
    for n in range (SMA_len, len(data)-return_len-np.mod(len(data)-SMA_len, return_len),return_len): 
        #the stopping point maximises the loops us of dtaset without falling outside of it.
        price = data.iloc[n]
        next_price = data.iloc[n+return_len]
        SMA = data.iloc[n-SMA_len:n].mean()
        acct_value.append(timing)
        if (price >= SMA) & (hold =='stock'): #the first case for when the loop checks for a price signal
            timing = next_price/price*timing
        elif (price >= SMA) & (hold =='cash'):#the second case for when the loop checks for a price signal
            hold = 'stock'
            open_position = price
            timing = next_price/price*timing
        elif (price < SMA) & (hold =='stock'):
            #the third and final case for when the loop checks for a price signal else nothing happens.
            hold ='cash'
            close_position = price
            if close_position > open_position:
                trades.append(1)
            else: 
                trades.append(0)
    return(timing,acct_value,trades)

#The SMA function,includes current day included SMA. basic form is to compare price to SMA of length set by user.
#This function looks for another price signal after a number of days equal to the return length

In [1]:
# total daily contract volume

def daily_call_volume(ticker):
    day_vol = 0
    exp_dates = options.get_expiration_dates(ticker)
    for dates in exp_dates:
        calls = options.get_calls(ticker,dates)["Volume"].values
        calls = np.where(calls == '-', 0, calls)
        day_vol += calls.astype('int').sum()

In [ ]:
#old and defunct but still used in some project
def conditional_relative_performance(df,select_intervals=[.02,.01,.005,0,-.005,-.01,-.02]):
    #select_intervals = [.02,.01,0,-.01,-.02] #highest to lowest
    outperformance = np.zeros(len(select_intervals)+1)
    probabilities = np.zeros(len(select_intervals)+1)
    outperformance[0] = df[df['SPY']>=select_intervals[0]].mean().diff()[1]
    probabilities[0] = len(df[df['SPY']>=select_intervals[0]])/len(df)

    for i in range(1,len(select_intervals)):
        outperformance[i] = df[(df['SPY']<select_intervals[i-1]) & (df['SPY']>select_intervals[i])].mean().diff()[1]
        probabilities[i] = len(df[(df['SPY']<select_intervals[i-1]) & (df['SPY']>select_intervals[i])])/len(df)
    
    outperformance[-1] = df[df['SPY']<=select_intervals[-1]].mean().diff()[1]
    probabilities[-1] = len(df[df['SPY']<=select_intervals[-1]])/len(df)


    print('Given SPY moves more than',select_intervals[0], 'SPYC outperformance is', np.round(outperformance[0],4), '    Probability:', np.round(probabilities[0],4))
    for i in range(len(outperformance)-2):
        print('{0: <16}'.format('Given SPY moves'), round(select_intervals[i+1],4),'to',round(select_intervals[i],4),'SPYC outperformance is',np.round(outperformance[i+1],4),'Probability:', np.round(probabilities[i+1],3) )
    print('Given SPY moves below ',select_intervals[-1], 'SPYC outperformance is', np.round(outperformance[-1],4),'    Probability:', np.round(probabilities[-1],4))  
    print('Expected outperformance: ', np.dot(probabilities,outperformance))

In [9]:
#function that returns allocation changes
def hrp_signal(weights, row, data, rebalance_period, formation_period):
    formation = data[row - formation_period : row - 1]
    cov,corr = formation.cov(),formation.corr()     #2) compute correl matrix 
    dist=correlDist(corr)                #3) cluster
    link=sch.linkage(dist,'single')
    sortIx=getQuasiDiag(link)
    sortIx=corr.index[sortIx].tolist()   # recover labels
    df0=corr.loc[sortIx,sortIx]          # reorder
    hrp=getRecBipart(cov,sortIx)         #allocation
    hrp.index=data.columns
    return np.array(hrp)

#signal col names should be a list
def backtest_w_signals(data, signal_col_names,rebalance_period, formation_period, hrp_converge_rebals, ivp_converge_rebal,RebalanceOffset=0): 
    stocks = data.iloc[:,~data.columns.isin(signal_col_names)]
    signals = data.iloc[:,data.columns.isin(signal_col_names)]
    wealth = np.ones(int(len(stocks.T.index)))/len(stocks.T.index)
    wealth_arr = [1]
    consecutive_turbulant_days = 0
    
    for n in range (formation_period, len(stocks)): #shorten for testing
        for i in range(len(wealth)):
            wealth[i]=wealth[i]*(1+stocks.iloc[n,i])
        total_wealth = sum(wealth)
        weights = np.array(wealth)/total_wealth
        
            
        if signals['Turbulance'][n]>(np.exp(-.1*(n-100))+2.2)*signals['Turbulance'][:n].std(): #exponential decay was created from plotting the function
            consecutive_turbulant_days+=1
            if consecutive_turbulant_days>=2:    
                formation = stocks[n - formation_period : n - 1]
                cov = formation.cov()
                ivp_weights = getIVP(cov)
                inverse_ivp_rebal = -(ivp_weights - weights)
                inv_ivp_scaled = inverse_ivp_rebal/ivp_converge_rebal
                
                #if rebalance would push an asset negative then split the excess subtraction between the other assets
                test = weights + inv_ivp_scaled
                scaling_array = np.zeros([len(weights)]) #sets hard boundary to keep this a long only portfolio
                for asset in range(len(test)):
                    if test[asset] >=0:
                        pass
                    while test[asset]<0:
                        test[asset] = test[asset]+.001
                        scaling_array[asset] = scaling_array[asset]+.001
                        if test[asset]>=0:
                            break
                amount_of_scaling = sum(scaling_array)/len(np.argwhere(scaling_array==0))
                
                for locations in np.argwhere(scaling_array==0):
                    test[locations]=test[locations]-amount_of_scaling #flipping this negative sign adds leverage which works great!
                inverse_ivp_rebal = test - weights
        else:
            inverse_ivp_rebal = weights * 0
            consecutive_turbulant_days = 0
            

        if ((n+RebalanceOffset) % rebalance_period == 0):   #rebalance
            hrp_weights = hrp_signal(weights, n, stocks, rebalance_period, formation_period)
            hrp_rebal = (hrp_weights - weights) / hrp_converge_rebals
        else:
            hrp_rebal = weights * 0
            
        #then rebalance by adding signals together
        weights_post_rebal = weights + inverse_ivp_rebal + hrp_rebal
        
        for asset in range(len(wealth)):
            wealth[asset] = total_wealth * weights_post_rebal[asset]

        wealth_arr.append(sum(wealth))
    wealth_arr.append(sum(wealth))
    
    return(wealth_arr)

#wrapper to call backtest with signals function and control for RebalanceTimingLuck
def backtest_w_RBL(*args):
    number_of_iterations = np.floor(np.array(args[2])/5)
    array_pre_average = np.zeros([len(args[0])-args[3]+2,int(number_of_iterations)]) #plus two comes from my formulation of backtesting function
    for n in range(int(number_of_iterations)):
        array_pre_average[:,n] = backtest_w_signals(args[0],args[1],args[2],args[3],args[4],args[5],RebalanceOffset=n*5)
        #take average of backtests 
    return(array_pre_average.mean(axis=1))



In [1]:
from sklearn.model_selection import learning_curve
def plot_learning_curve(
    estimator,
    X,
    y,
):

  train_sizes, train_scores, test_scores = learning_curve(
        estimator,
        X = X,
        y = y,
        scoring='neg_mean_squared_error',
        cv=5,
    )
  train_scores_mean = -np.mean(train_scores, axis=1)

  train_scores_std = np.std(train_scores, axis=1)

  test_scores_mean = -np.mean(test_scores, axis=1)

  test_scores_std = np.std(test_scores, axis=1)


  _, axes = plt.subplots(1, 1, figsize=(20, 5))
  axes.set_xlabel("Training examples")
  axes.set_ylabel("Score")

  axes.grid()
  axes.fill_between(
        train_sizes,
        train_scores_mean - train_scores_std,
        train_scores_mean + train_scores_std,
        alpha=0.1,
        color="r",
    )
  axes.fill_between(
        train_sizes,
        test_scores_mean - test_scores_std,
        test_scores_mean + test_scores_std,
        alpha=0.1,
        color="g",
    )
  axes.plot(
        train_sizes, train_scores_mean, "o-", color="r", label="Training score"
    )
  axes.plot(
        train_sizes, test_scores_mean, "o-", color="g", label="Cross-validation score"
    )
  axes.legend(loc="best")
  print(test_scores_mean[-1])
  print(train_scores_mean[-1])

In [5]:
#give pandas series for x and y
def time_series_processing(x,y,sequence_len):
    ts_processing = tf.keras.preprocessing.timeseries_dataset_from_array(
    x[:-sequence_len],
    y[sequence_len:],
    sequence_length=sequence_len, 
    sampling_rate=1,
    batch_size=len(x),
    sequence_stride=1)
    
    for batch in ts_processing.take(1):
        batched_inputs, targets = batch
    return batched_inputs.numpy(), targets.numpy()